# Physics-Adherence (PA) 策略详解

## 策略核心思想
Physics-Adherence查询策略通过量化预测的速度场对Navier-Stokes方程的遵守程度来评估未标记样本的质量。违反物理定律越严重的预测，其不确定性越高，越值得被标注。

## 两个核心公式解析

### 1. 连续性方程 (Continuity Equation)
```
Lcontinuity(xi) := mean over p∈Pi |（∇ · yi）p|
```

**物理含义：**
- **∇ · yi** 表示速度场的散度（divergence）
- 对于不可压缩流体，质量守恒要求 **∇ · **v** = 0**（散度为零）
- 这个公式计算预测速度场散度的绝对值平均

**违反情况：**
- 如果 Lcontinuity 值很大 → 预测的流体"凭空产生"或"凭空消失"
- 违反了质量守恒定律
- 表明模型对该样本的预测不可靠

### 2. 动量方程 (Momentum Equation)  
```
Lmomentum(xi) := mean over p∈Pi ∥ρ((yi · ∇)yi)p − µ(∆yi)p∥2
```

**物理含义：**
- **ρ((yi · ∇)yi)**: 惯性项，描述流体加速度
- **µ(∆yi)**: 粘性项，描述粘性力的作用  
- 这两项应该平衡（加上压力梯度项）

**违反情况：**
- 如果 Lmomentum 值很大 → 惯性力和粘性力严重不平衡
- 违反了动量守恒定律
- 表明预测的流场动力学不合理

## 综合评分
```
LNS = Lcontinuity + λ × Lmomentum
```
其中 λ = 1×10⁻⁴ 是权重参数，用于平衡两项的重要性。

**PA策略选择逻辑：**
- LNS值越大 → 物理违反越严重 → 模型越不确定 → 越应该被标注

# 梯度算子详解：正三角∇ vs 倒三角∆

## 数学基础

### 1. 正三角 ∇ (Nabla算子)
**∇** 是**梯度算子**，在三维空间中定义为：
```
∇ = (∂/∂x, ∂/∂y, ∂/∂z)
```

### 2. 倒三角 ∆ (拉普拉斯算子) 
**∆** 是**拉普拉斯算子**，定义为：
```
∆ = ∇² = ∂²/∂x² + ∂²/∂y² + ∂²/∂z²
```

## 在PA策略中的具体应用

### 连续性方程中的散度 (∇·**v**)
    "对于速度场 **v** = (vₓ, vᵧ, vᵤ)：
",
```
∇·**v** = ∂vₓ/∂x + ∂vᵧ/∂y + ∂vᵤ/∂z
```
- **物理意义**：流体在某点的"源汇强度"
- **∇·v > 0**：流体在此点产生（源）
- **∇·v < 0**：流体在此点消失（汇）  
- **∇·v = 0**：流体既不产生也不消失（连续性）

### 动量方程中的拉普拉斯 (∆**v**)
对于速度场的每个分量：
```
∆vₓ = ∂²vₓ/∂x² + ∂²vₓ/∂y² + ∂²vₓ/∂z²
∆vᵧ = ∂²vᵧ/∂x² + ∂²vᵧ/∂y² + ∂²vᵧ/∂z²
∆vᵤ = ∂²vᵤ/∂x² + ∂²vᵤ/∂y² + ∂²vᵤ/∂z²
```
- **物理意义**：粘性扩散的强度
- **μ∆v**：粘性力项，描述分子粘性对流动的影响

### 惯性项 (**v**·∇)**v**
这是一个**对流算子**：
```
(**v**·∇)vₓ = vₓ(∂vₓ/∂x) + vᵧ(∂vₓ/∂y) + vᵤ(∂vₓ/∂z)
(**v**·∇)vᵧ = vₓ(∂vᵧ/∂x) + vᵧ(∂vᵧ/∂y) + vᵤ(∂vᵧ/∂z)
(**v**·∇)vᵤ = vₓ(∂vᵤ/∂x) + vᵧ(∂vᵤ/∂y) + vᵤ(∂vᵤ/∂z)
```
- **物理意义**：流体粒子沿流线的加速度

# PA策略的壁面量改写：基于压力和剪切应力

## 工程实际需求
在流体工程中，我们通常更关心：
- **壁面压力分布** p(wall)：影响结构载荷
- **壁面剪切应力** τ(wall)：影响传热传质和摩擦阻力

## 原始PA公式回顾
```
LNS = ||∇·**v**||₁ + λ||ρ(**v**·∇)**v** - μ∇²**v**||₂
```

## 壁面量改写策略

### 1. 连续性方程的壁面改写

**原始形式**：∇·v = 0（体积守恒）

**壁面改写**：基于**质量通量平衡**
```
L_mass_balance = |∫_wall (ρ**v**·**n**) dS|
```
其中：
- ρ**v**·**n**：壁面法向质量通量
- 对于固体壁面：**v**·**n** = 0（无穿透条件）
- 对于多孔壁面：∑入口通量 = ∑出口通量

**物理意义**：
- 检查壁面边界条件是否满足
- 验证进出口质量平衡

### 2. 动量方程的壁面改写

**原始形式**：ρ(v·∇)v = μ∇²v + ∇p（动量平衡）

**壁面改写**：基于**力平衡方程**
```
L_force_balance = ||∫_wall τ_wall dS - ∫_wall p·**n** dS - F_external||₂
```

其中：
- **τ_wall = μ(∂**v**/∂n)|wall**：壁面剪切应力
- **p·**n****：壁面压力的法向分量  
- **F_external**：外部体力（重力、浮力等）

### 3. 实用的壁面PA公式

**改写后的PA策略**：
```
LNS_wall = α·L_pressure + β·L_shear + γ·L_boundary
```

**详细展开**：

#### A. 压力一致性检验
```
L_pressure = ||p_predicted - p_physics||₂
```
其中 p_physics 可以通过伯努利方程或压力梯度估算：
```
p_physics ≈ p₀ + ½ρv² + ρgh  (伯努利方程)
```

#### B. 剪切应力检验  
```
L_shear = ||τ_predicted - τ_physics||₂
```
其中：
```
τ_physics = μ(∂**v**/∂n)|wall ≈ μ(**v**_bulk/δ)  (边界层近似)
```
- δ：边界层厚度
- **v**_bulk：主流速度

#### C. 边界条件检验
```
L_boundary = ||**v**_wall - **v**_boundary_condition||₂
```
- 无滑移壁面：**v**_wall = 0
- 滑移壁面：**v**_tangential = f(τ_wall)

## 实际工程优势

### 1. 计算效率
- 只需壁面数据，不需要整个流场
- 减少计算量和存储需求
- 适合高雷诺数流动

### 2. 物理直观性
- 直接对应工程关心的物理量
- 便于与实验数据对比验证
- 易于设置物理合理的阈值

### 3. 边界条件导向
- 重点关注边界条件的满足程度
- 适合复杂几何的壁面约束问题
- 能捕捉边界层分离等关键现象

# 质量通量平衡公式详解

## 公式修正与解释

您看到的质量通量平衡公式：
```
L_mass_balance = |∫_wall (ρ**v**·**n**) dS|
```

### 符号说明（修正显示问题）

**可能的显示问题**：
- 如果您看到"1乱码"，这通常是因为：
  - **向量箭头 v** 在某些字体中显示为数字"1"
  - **点积符号 ·** 可能显示异常
  - **积分符号 ∫** 在部分环境中显示不正确

**正确的数学表达**：
```
L_mass_balance = |∫_wall (ρ * **v** · **n**) dS|
```
或者用更清晰的ASCII表示：
```
L_mass_balance = |∫_wall (rho * v_vec · n_vec) dS|
```

### 各符号的物理含义

1. **ρ (rho)**：流体密度 [kg/m³]
2. **v (v_vec)**：速度矢量 [m/s]
   - 三维形式：**v** = (vx, vy, vz)
3. **n (n_vec)**：壁面外法向量（单位向量）
   - 指向流体外部的方向
4. **v·n (点积)**：速度在法向的分量
   - **v**·**n** = vx·nx + vy·ny + vz·nz
5. **dS**：壁面微元面积
6. **∫_wall**：沿整个壁面积分

### 物理意义详解

**质量通量密度**：ρ**v**·**n**
- 单位面积上单位时间通过的质量
- 单位：[kg/(m²·s)]

**总质量通量**：∫_wall (ρ**v**·**n**) dS
- 整个壁面的总质量流率
- 单位：[kg/s]

### 边界条件的具体表现

#### 1. 固体壁面（无穿透条件）
```
**v**·**n** = 0  →  ρ**v**·**n** = 0
```
- 流体不能穿透固体壁面
- 法向速度分量为零
- 但切向速度可以非零（剪切流动）

#### 2. 入口边界
```
**v**·**n** < 0  （流入）
∫_inlet (ρ**v**·**n**) dS = -ṁ_in  （负值表示流入）
```

#### 3. 出口边界  
```
**v**·**n** > 0  （流出）
∫_outlet (ρ**v**·**n**) dS = +ṁ_out  （正值表示流出）
```

#### 4. 质量守恒要求
```
∑ ṁ_in = ∑ ṁ_out
即：|∫_inlet (ρ**v**·**n**) dS| = |∫_outlet (ρ**v**·**n**) dS|
```

## 实际计算示例

### 简化的矩形管道
假设管道截面为矩形，入口速度均匀：

**入口**：
- 面积：A_in = W × H
- 速度：**v**_in = (U, 0, 0)  
- 法向量：**n**_in = (-1, 0, 0)
- 质量流率：ṁ_in = ρ × U × A_in

**出口**：
- 面积：A_out = W × H
- 速度：**v**_out = (U_out, 0, 0)
- 法向量：**n**_out = (1, 0, 0)  
- 质量流率：ṁ_out = ρ × U_out × A_out

**质量守恒检验**：
```
L_mass_balance = |ṁ_in - ṁ_out| = |ρUA_in - ρU_out A_out|
```

对于不可压缩流体且管道等截面：
- A_in = A_out = A
- 质量守恒要求：U = U_out
- 理想情况：L_mass_balance = 0

## PA策略中的应用

在壁面PA策略中，这个公式用于：

1. **检验边界条件合理性**
   - 固体壁面是否满足无穿透条件
   - 进出口流量是否平衡

2. **识别物理违反样本**
   - L_mass_balance 值大 → 质量守恒违反严重
   - 这类样本值得优先标注学习

3. **替代原始散度计算**
   - 从体积积分（∇·v）转为面积分（边界通量）
   - 计算效率更高，物理意义更直观

# 矢量表示法说明

## 📝 符号约定修正

为了避免显示乱码问题，本notebook中所有矢量都采用**粗体**表示：

### 矢量符号对照表

| 标准数学记号 | 本notebook表示 | 物理含义 |
|-------------|----------------|----------|
| v⃗ | **v** | 速度矢量 |
| n⃗ | **n** | 法向量 |
| F⃗ | **F** | 力矢量 |
| ∇ | ∇ | 梯度算子（标量算子） |

### 常用运算表示

| 数学运算 | 标准记号 | 本notebook表示 | 说明 |
|---------|---------|----------------|------|
| 点积 | v⃗·n⃗ | **v**·**n** | 标量结果 |
| 散度 | ∇·v⃗ | ∇·**v** | 标量结果 |
| 梯度 | ∇f | ∇f | 矢量结果 |
| 拉普拉斯 | ∇²v⃗ | ∇²**v** | 矢量结果 |
| 对流项 | (v⃗·∇)v⃗ | (**v**·∇)**v** | 矢量结果 |

### PA策略公式总览（粗体版本）

#### 原始公式
```
LNS = ||∇·**v**||₁ + λ||ρ(**v**·∇)**v** - μ∇²**v**||₂
```

#### 壁面改写公式
```
LNS_wall = α·L_pressure + β·L_shear + γ·L_boundary
```

其中：
- `L_pressure = ||p_predicted - p_physics||₂`
- `L_shear = ||τ_predicted - τ_physics||₂`  
- `L_boundary = ||**v**_wall - **v**_boundary_condition||₂`

#### 质量通量公式
```
L_mass_balance = |∫_wall (ρ**v**·**n**) dS|
```

### 优势说明

✅ **显示稳定**：粗体在所有环境中都能正确显示  
✅ **含义清晰**：粗体明确表示矢量性质  
✅ **易于识别**：与标量符号形成鲜明对比  
✅ **通用兼容**：适用于各种编辑器和显示设备